# Redes Neurais

Nesta seção vamos treinar uma rede neural simples, implementada com PyTorch, para classificar pontos em 2D de um dataset sintético.

In [ ]:
import numpy as np
from sklearn.datasets import make_moons
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

## Datasets e DataLoaders

Para treinar um modelo, precisamos de um pipeline de dados eficiente. O PyTorch oferece duas primitivas de dados fundamentais para isso: `torch.utils.data.Dataset` e `torch.utils.data.DataLoader`.

In [ ]:
# Gerar dataset sintético (moon dataset)
X, y = make_moons(n_samples=1000, noise=0.2, random_state=seed)
X = X.astype(np.float32)
y = y.astype(np.int64)

In [ ]:
plt.figure(figsize=(6, 5))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap="coolwarm", edgecolor=None, s=25)
plt.title("Dataset Sintético: Two Moons")
plt.xlabel("x1")
plt.ylabel("x2")
plt.grid(True, alpha=0.3)
plt.show()

### O Dataset

A classe `Dataset` é uma classe abstrata que representa uma fonte de dados. Para criar seu próprio dataset, você precisa herdar desta classe e sobrescrever três métodos especiais (métodos mágicos):

1.  `__init__(self, ...)`: O construtor da classe. É executado uma única vez ao instanciar o dataset. É aqui que você normalmente faria o carregamento inicial dos dados (ex: ler um arquivo CSV, encontrar os caminhos das imagens em um diretório).

2.  `__len__(self)`: Este método deve retornar o número total de amostras no seu dataset. O `DataLoader` utiliza essa informação para saber o tamanho do dataset e definir os índices.

3.  `__getitem__(self, idx)`: Este método é responsável por carregar e retornar **uma única amostra** do dataset, dado um índice `idx`. É aqui que transformações nos dados (como data augmentation ou normalização) são frequentemente aplicadas.

In [ ]:
class MoonsDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
full_dataset = MoonsDataset(X, y)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(seed)
)

print(f"Tamanho treino: {len(train_dataset)}, validação: {len(val_dataset)}")

In [ ]:
X, y = train_dataset[0]
X.shape, y

### O Data Loader

Uma vez que temos um objeto `Dataset`, que sabe como acessar amostras individuais, precisamos de uma forma eficiente de iterar sobre ele durante o treinamento. É aqui que entra o `DataLoader`.

O `DataLoader` é um iterador que envolve um `Dataset` e automatiza o processo de criação de mini-lotes (*mini-batches*). Suas principais funcionalidades são:

-   **Agrupamento em Lotes (Batching)**: Agrupa múltiplas amostras retornadas pelo `__getitem__` do `Dataset` para formar um lote (batch) de dados.
-   **Embaralhamento (Shuffling)**: Permite embaralhar os dados a cada época (`shuffle=True`) para evitar que o modelo aprenda a ordem dos dados e melhore a generalização.
-   **Carregamento Paralelo (Parallel Loading)**: Pode usar múltiplos subprocessos (`num_workers`) para carregar os dados em paralelo, evitando que o carregamento de dados se torne um gargalo.

In [ ]:
batch_size = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
X, y = next(iter(train_loader))
X.shape, y.shape

## Modelo

Vamos definir uma rede neural totalmente conectada simples (MLP) para classificar os pontos 2D em duas classes.

- Entrada: vetor 2D (coordenadas do ponto).
- Camadas escondidas: algumas camadas lineares com não-linearidade ReLU.
- Saída: logits para 2 classes.

In [ ]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=16, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
model = MLPClassifier().to(device)

## Treinamento

No treinamento vamos:

- Iterar por várias épocas.
- Para cada época:
  - Colocar o modelo em modo de treino (`model.train()`).
  - Percorrer o `train_loader`, calculando *loss* e atualizando os pesos com `optimizer.step()`.
  - Ao final da época, avaliar no conjunto de validação para obter *loss* e acurácia de validação.
- Armazenar o histórico de *loss* e acurácia de treino/validação, e ao final plotar as curvas.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [ ]:
num_epochs = 30
history = {k: [] for k in ["train_loss","val_loss","train_acc","val_acc"]}

for epoch in range(1, num_epochs + 1):

    # ---------- Treino ----------
    model.train()
    train_losses, train_accs = [], []

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        preds = logits.argmax(1)
        acc = (preds == yb).float().mean().item()

        train_losses.append(loss.item())
        train_accs.append(acc)

    train_loss = np.mean(train_losses)
    train_acc = np.mean(train_accs)

    # ---------- Validação ----------
    model.eval()
    val_losses, val_accs = [], []

    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)

            logits = model(xb)
            loss = criterion(logits, yb)

            preds = logits.argmax(1)
            acc = (preds == yb).float().mean().item()

            val_losses.append(loss.item())
            val_accs.append(acc)

    val_loss = np.mean(val_losses)
    val_acc = np.mean(val_accs)

    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["train_acc"].append(train_acc)
    history["val_acc"].append(val_acc)

    if epoch % 3 == 0 or epoch == 1:
        print(f"{epoch:03d} | loss {train_loss:.4f}/{val_loss:.4f}  acc {train_acc:.3f}/{val_acc:.3f}")

In [ ]:
epochs = range(1, num_epochs + 1)
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs, history["train_loss"], label="Treino")
plt.plot(epochs, history["val_loss"], label="Validação")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.title("Curva de Loss")
plt.legend()

# Plotar curvas de acurácia
plt.subplot(1, 2, 2)
plt.plot(epochs, history["train_acc"], label="Treino")
plt.plot(epochs, history["val_acc"], label="Validação")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.title("Curva de Acurácia")
plt.legend()

plt.tight_layout()
plt.show()

# Validação

Aqui calculamos explicitamente a acurácia final no conjunto de validação utilizando o modelo treinado. Essa etapa é semelhante ao que fizemos ao longo do treinamento, mas executada apenas uma vez ao final, como resumo.

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        logits = model(xb)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

val_accuracy_final = correct / total
print(f"Acurácia final na validação: {val_accuracy_final:.3f}")

In [ ]:
def plot_decision_boundary(model, X, y, device):
    model.eval()
    X_np = X if isinstance(X, np.ndarray) else X.numpy()
    y_np = y if isinstance(y, np.ndarray) else y.numpy()

    x_min, x_max = X_np[:, 0].min() - 0.5, X_np[:, 0].max() + 0.5
    y_min, y_max = X_np[:, 1].min() - 0.5, X_np[:, 1].max() + 0.5

    # Grade de pontos
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 200),
        np.linspace(y_min, y_max, 200)
    )
    grid = np.c_[xx.ravel(), yy.ravel()].astype(np.float32)
    grid_t = torch.from_numpy(grid).to(device)

    with torch.no_grad():
        logits = model(grid_t)
        probs = F.softmax(logits, dim=1)[:, 1]  # prob da classe 1
        Z = probs.cpu().numpy().reshape(xx.shape)

    plt.figure(figsize=(6, 5))
    # Contorno preenchido com as probabilidades
    cs = plt.contourf(xx, yy, Z, levels=50, alpha=0.8, cmap="RdBu")
    plt.colorbar(cs, label="Probabilidade da classe 1")

    # Pontos reais
    plt.scatter(X_np[:, 0], X_np[:, 1], c=y_np, cmap="viridis", edgecolor="k", s=20)
    plt.title("Fronteira de decisão da rede neural")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.show()

# Plotar fronteira usando todo o dataset
plot_decision_boundary(model, X, y, device)

## Inferência

Na inferência usamos o modelo treinado para prever a classe de novos pontos 2D.

In [ ]:
# Exemplo de inferência em alguns pontos novos
novos_pontos = np.array([
    [-1.0, 0.5],
    [2.0, -0.5],
    [0.0, 1.0],
    [1.5, 0.0]
], dtype=np.float32)

novos_pontos_t = torch.from_numpy(novos_pontos).to(device)

model.eval()
with torch.no_grad():
    logits = model(novos_pontos_t)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1).cpu().numpy()

for p, c in zip(novos_pontos, preds):
    print(f"Ponto {p} -> classe prevista: {c}")